# Check for missing instrument fields

You can run the cells below directly in LUSID's JupyterHub.

The `%%luminesce` is a magic command which passes the cell query string to Lumipy,
which then returns a DataFrame.
    

#### Step 1: Create instrument properties

In [ ]:
%%luminesce

-- ===============================================================
-- Description:
-- In this query, make properties available to Luminesce.
-- We create instrument properties needed for the QC
-- See the following page for further details:
-- https://support.lusid.com/knowledgebase/article/KA-01702/en-us
-- ===============================================================
-- 1. Define new properties HERE:
@newProperties = values
('Instrument/ibor/QualityControlStatus', 'Text', 'QualityControlStatus', 'The quality control status of the instrument.'),
('Instrument/ibor/Sector', 'Text', 'Sector', 'The sector that the instrument belongs to.'),
('Instrument/ibor/SourceFile', 'Text', 'SourceFile', 'The source file of the instrument.'),
('Instrument/ibor/InternalRating', 'Text', 'InternalRating', 'The internal rating of the instrument.'),
('Instrument/ibor/SharesOutstanding', 'Decimal', 'SharesOutstanding', 'The number of shares outstanding for the instrument.'),
('Instrument/ibor/RegFlag', 'Text', 'RegFlag', 'Red flag.'),
('Instrument/ibor/MissingFields', 'Text', 'MissingFields', 'Missing fields.');

-- 2. Create a view of new properties to upload to LUSID
@property_definition = select
Column3 as [DisplayName],
'Instrument' as [Domain],
'ibor' as [PropertyScope],
Column3 as [PropertyCode],
'Property' as [ConstraintStyle],
'system' as [DataTypeScope],
case when Column2 == 'Text' then 'string' else 'number' end as [DataTypeCode]
from @newProperties;

-- 3. The results of writing the new property definitions can be seen from the query below:
select * from Lusid.Property.Definition.Writer
where ToWrite = @property_definition;


#### Step 2: Load instruments into lusid

In [ ]:
%%luminesce

-- ===============================================================
-- Description:
-- In this file, we load instruments into LUSID with the new
-- properties defined in the first file.
-- ===============================================================

-- Define file name, this is the instruments file on Drive

@@file_name = select 'equity_instruments_20220819.csv';

-- Load data from Drive

@instruments_data =
use Drive.Csv with @@file_name
--file=/luminesce-examples/{@@file_name}
--addFileName
enduse;

-- 2. Upload the instruments and their default properties into Lusid.Instrument.Equity provider

@equity_instruments =
select Name as DisplayName, ClientInternal as ClientInternal, ISIN as Isin, SEDOL as Sedol, Currency as DomCcy, 'MissingFields' as Scope
from @instruments_data;

@write_instruments =
select *
from Lusid.Instrument.Equity.Writer
where ToWrite = @equity_instruments
   and DeletePropertiesWhereNull = True;

-- 3. Create view of custom property values

@custom_props =
select ClientInternal as EntityId, Sector, InternalRating, SharesOutstanding, RegFlag, 'equity_instruments_20220819' as
   SourceFile, 'MissingFields: QC not started' as 'MissingFields', 'NotStarted' as QualityControlStatus
from @instruments_data;

@unpivoted =
use Tools.Unpivot with @custom_props
--key=EntityId
--keyIsNotUnique
enduse;

@instr_props =
select li.LusidInstrumentId as EntityId, 'LusidInstrumentId' as EntityIdType, 'Instrument' as Domain, 'ibor' as
   PropertyScope, a.ValueColumnName as PropertyCode, a.ValueText as Value, 'MissingFields' as EntityScope
from @unpivoted a
inner join Lusid.Instrument li
   on li.ClientInternal = a.EntityId
   where Scope = 'MissingFields';

-- 4. Upload custom properties data to Lusid.Property

@write_properties =
select *
from Lusid.Property.Writer
where ToWrite = @instr_props;


#### Step 3: Check for missing instrument fields

In [ ]:
%%luminesce

-- ===============================================================
-- Description:
-- In this file, we run a QC check to check for missing fields
-- ===============================================================

-- 1. Get values for custom properties and transform data.

@custom_props =
select InstrumentId, PropertyCode, Value
from Lusid.Instrument.Property p
where propertyscope = 'ibor'
   and propertycode in ('RegFlag', 'Sector', 'SharesOutstanding', 'InternalRating', 'SourceFile');

@pivoted =
use Tools.Pivot with @custom_props
--key=PropertyCode
--aggregateColumns=Value
enduse;

-- 2. Create view for instruments from source file with custom and default properties.

@data_qc =
select *
from @pivoted p
inner join (
   select DisplayName, Isin, ClientInternal, LusidInstrumentId, Sedol, DomCcy
   from Lusid.Instrument.Equity
   where Scope = 'MissingFields'
   ) q
   on q.LusidInstrumentId = p.InstrumentId
where SourceFile = 'equity_instruments_20220819';

-- 3. Run quality control check on data and populate `QualityControlStatus`

@qc_check =
select *, case
      when Sector is null
         then 'Failed'
      when RegFlag is null
         then 'Failed'
      when InternalRating is null
         then 'Failed'
      when SharesOutstanding is null
         then 'Failed'
      else 'Passed'
      end as 'QualityControlStatus'
from @data_qc;

-- 4. Filter for PASSED instruments and populate `MissingFields`

@qc_passed =
select *, 'Missing fields: None' as 'MissingFields'
from @qc_check
where QualityControlStatus = 'Passed';

-- 5. Filter for FAILED instruments and populate `MissingFields`

@qc_failed =
select *, 'Missing fields: ' || case
      when InternalRating is null
         then 'InternalRating, '
      else ''
      end || case
      when RegFlag is null
         then 'RegFlag, '
      else ''
      end || case
      when SharesOutstanding is null
         then 'SharesOutstanding, '
      else ''
      end || case
      when Sector is null
         then 'Sector, '
      else ''
      end as 'MissingFields'
from @qc_check
where QualityControlStatus = 'Failed';

-- 6. Create a view of all PASSED and FAILED instruments from source file, with `MissingFields` and `QualityControlStatus` properties.

@pass_and_fail =
select InstrumentId, MissingFields, QualityControlStatus
from @qc_failed
union all
select InstrumentId, MissingFields, QualityControlStatus
from @qc_passed;

@qc_props =
use Tools.Unpivot with @pass_and_fail
--key=InstrumentId
--keyIsNotUnique
enduse;

@props_towrite =
select InstrumentId as EntityId, 'LusidInstrumentId' as EntityIdType, 'Instrument' as Domain, 'ibor' as PropertyScope, ValueColumnName as PropertyCode, ValueText as Value, 'MissingFields' as EntityScope
from @qc_props;

-- 7. Upload `QualityControlStatus` and `MissingFields` property for each instrument to Lusid.Property provider.
-- Print results of writing data to console;

select *
from Lusid.Property.Writer
where ToWrite = @props_towrite;

